CG-Net.ipynb 
* Created by: Carter Lyons

Contains:
* Deep neural network implementation of Compound Gaussian Network (CG-Net) form by applying algorithm unrolling to the Compound Gaussian Least Squares (CG-LS) iterative algorithm. Estimates signal representation coefficients from linear measurements.
* For more information reference:
    1. Lyons C., Raj R. G., and Cheney M. (2023). "A Compound Gaussian Network for Solving Linear Inverse Problems," In-Review.
    2. Lyons C., Raj R. G., and Cheney M. (2022). "CG-Net: A Compound Gaussian Prior Based Unrolled Imaging Network," in *2022 IEEE Asia-Pacific Signal and Information Processing Association Annual Summit and Conference*, pp. 623-629.

Package Requirements (in addition to those required in Initialize.py):
* Tensorflow 2.5.0, pandas, matplotlib

In [1]:
# Import necessary python packages
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Import necessary additional code files
import Initialize
import LossFunctions
from CGNet import CGNet
from CGNet import Grad

# Create Network Structure

In [2]:
# Choose network parameters
img_size = 32  # Height and width of input images 
style = 'tridiag'  # Format of PD matrix in z quadratic descent direction. Options: 'tridiag', 'full'
angles = 15  # Number of uniformly spaced angles for Radon transform
noise = 60  # SNR of measurements (in dB)
num_training_samples = 20  # Number of training samples to be used
num_epochs, batch_size, TestBatchSize = 20, 20, 50  # Number of epochs for training, training batch size, and test batch size
K = 20  # Number of iterations to unroll
J = 1  # Number of steepest descent updates of z for each interation to unroll
scale = 1  # Parameter to scale the input measurements by
lamb, kappa, eta, eps = 0.3, 2, 0.5, 1e-3  # Initialization of regularization paramters lamb, kappa. Initialization of step size eta. Stabilizing paramter eps.
UseValidationData, ValidationSamples = True, 200  # Use validation data during training to check for overfitting. Number of samples to validate the network with.
# path_to_save = r'path_to_desired_save_folder'  # Path to save weights trained by the network.
path_to_save = r'C:\Users\LyonsC\Desktop\New folder'

n = img_size**2  # Signal size
B = Initialize.create_measurements('barbara.png', [[154, 154+img_size],[64,64+img_size]], import_phi = (False, [], 'bior1.1'), import_psi = (False, [], angles))  # From Initialize.py: Create Radon transform matrix. Create biorthogonal wavelet matrix. Use these matrices to form measurements of input 'Barbara' image
A = tf.constant(B.Psi@B.Phi, dtype = float)  # Measurement matrix: A = Psi*Phi (set dtype to float32)

In [3]:
# Network structure
model = CGNet(K, J, A, style, lamb, kappa = kappa, eta = eta, eps = eps, method = 'g', normalize = True)
model = model.call()

grad = Grad()
loss_fnc = LossFunctions.SSIM_loss(B.Phi, img_size, scale)
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)

In [ ]:
model.summary()  # Display summary of network structure (optional)

# Read in Train Data

In [5]:
path_to_train_data = r'Datasets\cifar10_{}angles_{}dB_train_data.pkl'.format(angles, noise)  # Path to training dataset
train_data = pd.read_pickle(path_to_train_data)  # Read in training dataset
size = len(train_data)
if UseValidationData:
    val_data = train_data.iloc[num_training_samples:min(size, num_training_samples+1000)]
    size = len(val_data)
    shuffled_data = val_data.sample(frac = 1)
train_data = train_data.iloc[:num_training_samples]  # Use only 'num_training_samples' from the dataset

# Network Training

In [ ]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
val_loss_results = []
training_time = []
results = []

train_data_size, val_data_size = len(train_data), len(val_data)
num_batches, num_val_batches = int(np.ceil(train_data_size/batch_size)), int(np.ceil(val_data_size/TestBatchSize))
print('Number of updates per epoch = ', num_batches)

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    val_loss_avg = tf.keras.metrics.Mean()
    shuffled_data = train_data.sample(frac = 1)
    start = time.time()
    for batch in range(num_batches):
        y = scale*np.array(shuffled_data.iloc[batch*batch_size:min([(batch+1)*batch_size,train_data_size])])  # Batch of train data
        c_act, y = y[:,np.shape(B.Psi)[0]:], y[:,:np.shape(B.Psi)[0]]  # Seperate into labels and measurements
        loss_value, grads = grad.call(model, loss_fnc, y, c_act)  # Compute forward and backward pass giving loss and gradients
        optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Apply network gradients using 'optimizer'
        epoch_loss_avg.update_state(loss_value)  # Update average loss for epoch
    training_time.append(time.time()-start)  # Saving batch training time
    train_loss_results.append(epoch_loss_avg.result())  # Update training loss over all epochs
    print("Training Time per Epoch: {:.3e} | Total Training Time: {:.3e}".format(sum(training_time)/(epoch+1), sum(training_time)))
    for batch in range(num_val_batches):
        y = scale*np.array(val_data.iloc[batch*TestBatchSize:min([(batch+1)*TestBatchSize,val_data_size])])
        c_act, y = y[:,np.shape(B.Psi)[0]:], y[:,:np.shape(B.Psi)[0]]
        val_loss = loss_fnc.call(c_act, model(y, training=False))
        val_loss_avg.update_state(val_loss)
    val_loss_results.append(val_loss_avg.result())
    results.append([epoch+1, epoch_loss_avg.result(), val_loss_avg.result(), sum(training_time)])
    print("Epoch {:02d}: Model Loss: {:.3e} | Test Loss: {:.3e}".format(epoch+1, epoch_loss_avg.result(), val_loss_avg.result()))
    if (epoch+1)%1==0:  # How often to save network weights and training results
        r = pd.DataFrame(np.array(results))
        r.to_csv(r'{}\training_loss_results.csv'.format(path_to_save)) 
        model.save_weights(r'{}\weights_epoch{}.h5'.format(path_to_save, epoch+1))  # Save network weights
        
model.save_weights(r'{}\weights.h5'.format(path_to_save))  # Save network weights
r = pd.DataFrame(np.array(results))
r.to_csv(r'{}\training_loss_results.csv'.format(path_to_save))  # Save training results